# Chapter 6 - Decision Tree Model

## Project: Credit Risk Score.

In [ ]:
# Importing Python Packages. 

# Data Manipulation. 
import pandas as pd 
import numpy as np

# Data Visualization Packages. 
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Gathering Package. 
import wget 

# Other
%matplotlib inline